# Importing Libraries

In [73]:
import pandas as pd
from dash import Dash, html, dcc, Input, Output, callback

import plotly.express as px

In [74]:
dataset = pd.read_excel("./data/dataset.xlsx");

In [75]:
dataset.head()

,product,price,quantity,total sales,date,region
0,pink morsel,3,546,1638,2018-02-06,north
1,pink morsel,3,549,1647,2018-02-06,south
2,pink morsel,3,577,1731,2018-02-06,east
3,pink morsel,3,519,1557,2018-02-06,west
4,pink morsel,3,529,1587,2018-02-07,north


# Preparing Datasets

In [76]:
df = pd.DataFrame(dataset)

In [77]:
df = df[['total sales', 'date', 'region']]
df.head()

,total sales,date,region
0,1638,2018-02-06,north
1,1647,2018-02-06,south
2,1731,2018-02-06,east
3,1557,2018-02-06,west
4,1587,2018-02-07,north


In [92]:
df_sales = pd.pivot_table(df, values = ['total sales'], index = 'date', aggfunc = np.sum, sort = True)
df_sales.head()

,total sales
date,
2018-02-06,6573
2018-02-07,6462
2018-02-08,6342
2018-02-09,6717
2018-02-10,6543


In [79]:
fig = px.line(df_sales, x = df_sales.index, y = 'total sales')

# Creating visualisation

In [80]:
# initialise app
app = Dash(__name__)

colors = {
    'background': '#111111',
    'text': '#000080'
}

# app layout
app.layout = html.Div([
    html.H1(
        children='Daily Total Sales [2018 - 2022]',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    
    dcc.Graph(
            figure = fig
    )
])

# run the app
if __name__ == '__main__':
    app.run(debug=True)

# Creating An Interactive Filter

In [95]:
# initialise app
app = Dash(__name__)

colors = {
    'background': '#111111',
    'text': '#000080'
}

# app layout
app.layout = html.Div([
    html.H1(
        children='Daily Total Sales by Region [2018 - 2022]',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    
    html.Div([
            dcc.Graph(figure = px.line(df, x = 'date', y = 'total sales'), id = 'my-output'),
            dcc.RadioItems(
                options=[
                {'label': 'North', 'value': 'north'},
                {'label': 'South', 'value': 'south'},
                {'label': 'East', 'value': 'east'},
                {'label': 'West', 'value': 'west'},
                {'label': 'All', 'value': 'all'}
            ],
                value='all', # default value
                id='my-input',
                inline=True
            )
        ], style={'width': '100%', 'float': 'center', 'display': 'inline-block'})
])


@callback(
    Output('my-output', 'figure'),
    Input('my-input', 'value'))
def update_graph(region):
    if region == 'all':
        filtered_df = df_sales
        fig = px.line(filtered_df, x = filtered_df.index, y = 'total sales')
    else:
        filtered_df = df[df['region'] == region]
        fig = px.line(filtered_df, x = 'date', y = 'total sales')

    fig.update_layout(transition_duration=500)

    return fig

# run the app
if __name__ == '__main__':
    app.run(debug=True)